## Análisis Preliminar de Evaluación de Impacto

**Hecho por**: Lina M. Ramírez 

**Fecha de inicio**: Abril 27, 2022

**Fecha de ultima actualización**: Abril 28, 2022

 - Este cuaderno tiene: 
    0. Manipulación de la base
    1. Características demográficas 
    2. Percepción sobre noticias 
    3. Percepción sobre tweets 
    4. Preguntas sobre desconfianza 
    
    
 - Pendiente: 
    -Hacer análisis temporal. 

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
import numpy as np
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats

In [2]:
%cd "/Users/bfiuser/Dropbox/Chicago/UChicago/dip/data"
df = pd.read_csv('Eval_04_27_2022.csv')

/Users/bfiuser/Dropbox/Chicago/UChicago/dip/data


In [170]:
pd.set_option('display.max_columns', 120)
pd.set_option('display.max_rows', 85)

In [171]:
#df.head(10)

In [172]:
#df.shape

Tenemos 74 variables, 3297 observaciones. 

In [173]:
df.columns

Index(['year_of_birth', 'gender', 'education_level', 'social_stratum',
       'nationality', 'department', 'municipality', 'id_last_four',
       'last_name_initial', 'email', 'form', 'score', 'video', 'fake_news_a',
       'fake_news_b', 'fake_news_c', 'fake_news_d', 'fake_news_e',
       'fake_news_f', 'fake_news_g', 'fake_news_h', 'fake_news_i',
       'fake_news_j', 'fake_news_k', 'fake_news_l', 'fake_news_m',
       'fake_news_n', 'fake_news_o', 'fake_news_p', 'fake_news_q',
       'fake_news_r', 'fake_news_s', 'tweet_a_lack_of_information',
       'tweet_a_report', 'tweet_b_lack_of_information', 'tweet_b_report',
       'headline_1', 'headline_2', 'headline_3', 'headline_4', 'headline_5',
       'headline_6', 'headline_7', 'headline_8', 'ie_confidence_a',
       'ie_confidence_b', 'ie_confidence_c', 'ie_confidence_d',
       'ie_confidence_e', 'ie_confidence_f', 'ie_confidence_g',
       'ie_confidence_h', 'ie_confidence_i', 'ie_discrimination_a',
       'ie_discrimination_b', 'i

## 0. Cleaning

### 0.1 General 

In [174]:
df['year_of_birth'].dtypes

dtype('int64')

In [175]:
df['age']=2022-df['year_of_birth']

In [176]:
# Grupos por genero, educacion, departamento, video y formulario. 
gender_grp = df.groupby(['gender'])
education_grp = df.groupby(['education_level'])
department_grp = df.groupby(['department'])
video_grp = df.groupby(['video'])
form_grp = df.groupby(['form'])

In [177]:
#renombrar columnas para facilidad
df.rename(columns={'ie_confidence_a':'trust_1', 'ie_confidence_b':'trust_2', 
                   'ie_confidence_c':'trust_3', 'ie_confidence_d':'trust_4',
                  'ie_confidence_e':'trust_5', 'ie_confidence_f':'trust_6',
                  'ie_confidence_g':'trust_7', 'ie_confidence_h':'trust_8', 
                   'ie_confidence_i' : 'trust_9'}, inplace = True)

### 0.2 Variables X (treatments) 

In [178]:
df["treatment"] = np.nan

In [179]:
#Para generar la variable con los 13 tratamientos: 
score_na = df['score'].isna()
score_no_na = df['score'].notna()
video_yes = df['video'] == 'Sí'
video_no = df['video'] =='No'
form_confianza = df['form'] == 'Confianza'
form_deshum = df['form'] == 'Deshumanización'
form_discri = df['form'] == 'Discriminización'
form_ambiguedad = df['form'] == 'Tolerancia a la ambigüedad'

df['treatment']  = np.select([score_na&video_no, score_na&video_yes&form_confianza, score_na&video_yes&form_deshum, 
                             score_na&video_yes&form_discri, score_na&video_yes&form_ambiguedad, 
                             score_no_na&video_no&form_confianza, score_no_na&video_no&form_deshum, 
                             score_no_na&video_no&form_discri, score_no_na&video_no&form_ambiguedad, 
                             score_no_na&video_yes&form_confianza, score_no_na&video_yes&form_deshum, 
                             score_no_na&video_yes&form_discri, score_no_na&video_yes&form_ambiguedad],
                             ['control','only_v_trust', 'only_v_dehumanization',
                              'only_v_discrimination', 'only_v_ambiguity', 'only_f_trust', 
                             'only_f_dehumanization', 'only_f_discrimination', 
                             'only_f_ambiguity', 'v+f_trust', 'v+f+dehumanization',
                             'v+f_discriminacion', 'v+f_ambiguity'])
                                    

In [180]:
df.sort_values(by=['treatment'], ascending=True, inplace=True)

In [181]:
#Grupos por tratamiento
treatment_grp = df.groupby(['treatment'])

In [182]:
#Base de datos con dummies para el tratamiento: 
df_dummies = pd.get_dummies(df, columns=['treatment'])
df_all_dummies = pd.get_dummies(df, columns=['treatment', 'gender', 'education_level', 
                                             'social_stratum', 'department'])

df2 = df_all_dummies
#df_dummies.columns


In [183]:
df2['any_treatment'] = np.nan
df2['any_form'] = 0
df2['any_video'] = 0
df2['any_video_form'] = 0

In [184]:
#Any treatment variable (cualquier tratamiento vs.control)
df.loc[df2['treatment_control'] == 1, 'any_treatment'] = 0 
df.loc[df2['treatment_control'] != 1, 'any_treatment'] = 1

In [185]:
#Any form
df.loc[df2['treatment_only_f_ambiguity'] == 1, 'any_form'] = 1 
df.loc[df2['treatment_only_f_dehumanization'] == 1, 'any_form'] = 1
df.loc[df2['treatment_only_f_discrimination'] == 1, 'any_form'] = 1
df.loc[df2['treatment_only_f_trust'] == 1, 'any_form'] = 1

In [186]:
# Any video 
df.loc[df2['treatment_only_v_ambiguity'] == 1, 'any_video'] = 1 
df.loc[df2['treatment_only_v_dehumanization'] == 1, 'any_video'] = 1
df.loc[df2['treatment_only_v_discrimination'] == 1, 'any_video'] = 1
df.loc[df2['treatment_v+f_trust'] == 1, 'any_video'] = 1

In [187]:
# Any video any form 
df.loc[df2['treatment_v+f+dehumanization'] == 1, 'any_video_form'] = 1 
df.loc[df2['treatment_v+f_ambiguity'] == 1, 'any_video_form'] = 1
df.loc[df2['treatment_v+f_discriminacion'] == 1, 'any_video_form'] = 1
df.loc[df2['treatment_only_v_trust'] == 1, 'any_video_form'] = 1

### 0.3 Variables Y (outcomes) 

In [188]:
#Promedio de respuestas a las preguntas de noticias falsas. 
df['fake_right'] = (df['fake_news_i'] + df['fake_news_j'] + df['fake_news_f'] + + df['fake_news_r'])/4
df['fake_left'] = (df['fake_news_d'] + df['fake_news_g'] + df['fake_news_h'] + df['fake_news_o'])/4
df['fake_center'] = (df['fake_news_a'] + df['fake_news_m'])/2
df['true_right'] = (df['fake_news_s'] + df['fake_news_e'] + df['fake_news_b'])/3
df['true_left'] = (df['fake_news_c'] + df['fake_news_l'] + df['fake_news_q'])/3
df['true_center'] = (df['fake_news_k'] + df['fake_news_p'] + df['fake_news_n'])/3

In [189]:
# Para diferencia de medias utilizando la variable 'video'
no = df[df['video']=='No']
yes = df[df['video']=='Sí']

In [190]:
# Para diferencia de medias utilizando la variable 'education_level'
posgrado = df[df['education_level']=='Posgrado']
primaria = df[df['education_level']=='Primaria']

### 0.4 Variables for Regression Model 

In [191]:
#REGRESSION MODEL 4. Base category control 
X = df2[['treatment_only_f_ambiguity', 'treatment_only_f_dehumanization',
       'treatment_only_f_discrimination', 'treatment_only_f_trust',
       'treatment_only_v_ambiguity', 'treatment_only_v_dehumanization',
       'treatment_only_v_discrimination', 'treatment_only_v_trust',
       'treatment_v+f+dehumanization', 'treatment_v+f_ambiguity',
       'treatment_v+f_discriminacion', 'treatment_v+f_trust']] 
X2 = sm.add_constant(X)

/Users/bfiuser/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


## 1. Características demográficas

- Edad
- Género 
- Nivel Educativo 
- Estrato Social
- Nacionalidad Departamento

### 1.1 Edad:

In [192]:
df['age'].describe()

count    3297.000000
mean       32.618744
std        11.519029
min        18.000000
25%        24.000000
50%        29.000000
75%        38.000000
max        99.000000
Name: age, dtype: float64

En promedio los que respondieron esta encuesta tienen 32.6 años.  El más joven tiene 18 años, y el más viejo 99 (puede ser un error). 

### 1.2 Género: 

In [193]:
df['gender'].value_counts(normalize=True)

Mujer     0.540795
Hombre    0.431604
Otro      0.027601
Name: gender, dtype: float64

El 54% de los participantes son mujeres. 


### 1.3 Nivel educativo: 

In [194]:
df['education_level'].value_counts(normalize=True)

Bachillerato             0.410373
Técnico / Tecnológico    0.295723
Título universitario     0.147103
Posgrado                 0.080983
Primaria                 0.065817
Name: education_level, dtype: float64

El 41% de los encuestados tienen bachillerato, el 30% tienen título técnico o tecnológico, el 15% tiene título universitario, el 8% tiene posgrado, y el 7% tiene primaria.  

### 1.4 Estrato Social:

In [195]:
df['social_stratum'].value_counts(normalize=True)

Estrato 2    0.383076
Estrato 3    0.243858
Estrato 1    0.222323
Estrato 4    0.085836
Estrato 5    0.030027
Estrato 6    0.028511
Rural        0.006369
Name: social_stratum, dtype: float64

El 38% de los encuestados son estrato 2. El 24% son estrato 3. 

### 1.5 Nacionalidad, departmento: 

In [196]:
df['nationality'].value_counts()

Colombia             2987
Venezuela              80
Anguilla               31
Angola                 29
Antártida              19
Samoa Americana        14
Armenia                14
Argelia                13
Andorra                11
Aruba                   8
Chile                   7
Argentina               7
Afganistán              5
Albania                 5
Antigua y Barbuda       5
Brasil                  4
Bolivia                 4
Azerbaiyán              3
Comoras                 3
Bahamas                 3
Baréin                  2
Botsuana                2
Bélgica                 2
Estados Unidos          2
Bermudas                2
Austria                 2
Belice                  2
Banglades               2
Camerún                 2
Cuba                    2
Canadá                  2
Australia               2
Cabo Verde              2
España                  1
Montenegro              1
China                   1
Líbano                  1
Santa Lucía             1
Dinamarca   

La mayoría de los encuestados son de Colombia (91%). 

In [197]:
df['department'].value_counts(normalize=True)

Bogotá D.C.                                                 0.559600
Atlántico                                                   0.118289
Cundinamarca                                                0.069457
Magdalena                                                   0.046406
Valle del Cauca                                             0.029724
Antioquia                                                   0.029724
Bolívar                                                     0.024871
Boyacá                                                      0.021838
Quindío                                                     0.014559
Meta                                                        0.010009
Risaralda                                                   0.009706
Santander                                                   0.008189
Caldas                                                      0.007583
Cesar                                                       0.006673
Huila                             

El 55% de los encuestados son de Bogotá. 

### 1.6 Tratamiento - tipo de test, puntaje y video

In [198]:
df['video'].value_counts(normalize=True)

No    0.621474
Sí    0.378526
Name: video, dtype: float64

La variable de video está desbalanceada. Solo el 38% de los participantes vieron video. 

In [199]:
df['form'].value_counts(normalize=True)

Confianza                     0.266297
Tolerancia a la ambigüedad    0.261184
Deshumanización               0.238176
Discriminización              0.234342
Name: form, dtype: float64

El test está balanceado entre los 4 tratamientos pero no entre los que ven video o no. 

In [200]:
df['treatment'].value_counts(normalize='True', sort=True)

control                  0.288141
only_f_trust             0.085836
only_f_ambiguity         0.083712
only_f_dehumanization    0.082803
only_f_discrimination    0.080983
v+f_ambiguity            0.054595
v+f_trust                0.053079
only_v_trust             0.050652
only_v_ambiguity         0.047619
v+f+dehumanization       0.046709
v+f_discriminacion       0.045496
only_v_discrimination    0.040340
only_v_dehumanization    0.040036
Name: treatment, dtype: float64

### 1.7 Balance en observables 

In [201]:
df_all_dummies.head(2)

,year_of_birth,nationality,municipality,id_last_four,last_name_initial,email,form,score,video,fake_news_a,fake_news_b,fake_news_c,fake_news_d,fake_news_e,fake_news_f,fake_news_g,fake_news_h,fake_news_i,fake_news_j,fake_news_k,fake_news_l,fake_news_m,fake_news_n,fake_news_o,fake_news_p,fake_news_q,fake_news_r,fake_news_s,tweet_a_lack_of_information,tweet_a_report,tweet_b_lack_of_information,tweet_b_report,headline_1,headline_2,headline_3,headline_4,headline_5,headline_6,headline_7,headline_8,trust_1,trust_2,trust_3,trust_4,trust_5,trust_6,trust_7,trust_8,trust_9,ie_discrimination_a,ie_discrimination_b,ie_discrimination_c,ie_discrimination_d,ie_discrimination_e,ie_discrimination_f,ie_ambiguity_tolerance_a,ie_ambiguity_tolerance_b,ie_ambiguity_tolerance_c,ie_ambiguity_tolerance_d,ie_ambiguity_tolerance_e,...,treatment_only_f_discrimination,treatment_only_f_trust,treatment_only_v_ambiguity,treatment_only_v_dehumanization,treatment_only_v_discrimination,treatment_only_v_trust,treatment_v+f+dehumanization,treatment_v+f_ambiguity,treatment_v+f_discriminacion,treatment_v+f_trust,gender_Hombre,gender_Mujer,gender_Otro,education_level_Bachillerato,education_level_Posgrado,education_level_Primaria,education_level_Técnico / Tecnológico,education_level_Título universitario,social_stratum_Estrato 1,social_stratum_Estrato 2,social_stratum_Estrato 3,social_stratum_Estrato 4,social_stratum_Estrato 5,social_stratum_Estrato 6,social_stratum_Rural,department_Amazonas,department_Antioquia,department_Arauca,"department_Archipiélago de San Andrés, Providencia y Santa Catalina",department_Atlántico,department_Bogotá D.C.,department_Bolívar,department_Boyacá,department_Caldas,department_Caquetá,department_Casanare,department_Cauca,department_Cesar,department_Chocó,department_Cundinamarca,department_Córdoba,department_Guainía,department_Guaviare,department_Huila,department_La Guajira,department_Magdalena,department_Meta,department_Nariño,department_Norte de Santander,department_Putumayo,department_Quindío,department_Risaralda,department_Santander,department_Sucre,department_Tolima,department_Valle del Cauca,any_treatment,any_form,any_video,any_video_form
0,1985,Belice,Bogotá D.C.,0,B,beatrivallejo@gmail.com,NaN,NaN,No,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,1,1,1,1,NaN,NaN,NaN,1.0,NaN,1.0,1.0,NaN,5,5,5,1,1,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0
970,1997,Colombia,Bogotá D.C.,8696,P,NaN,NaN,NaN,No,3,3,2,3,4,2,2,4,3,3,3,3,4,3,2,4,4,4,3,0,0,0,0,1.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,1,3,2,0,1,2,2,2,2,3,3,3,4,2,4,4,2,3,3,3,...,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0,0


In [202]:
Y_1 = df_all_dummies['gender_Mujer']
Y_2 = df_all_dummies['education_level_Posgrado']
Y_3 = df_all_dummies['education_level_Primaria']
Y_4 = df_all_dummies['social_stratum_Estrato 6']
Y_5 = df_all_dummies['social_stratum_Estrato 1']
Y_6 = df_all_dummies['department_Bogotá D.C.']

In [203]:
est = sm.OLS(Y_1, X2)
est2 = est.fit()
est2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           gender_Mujer   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.057
Date:                Fri, 29 Apr 2022   Prob (F-statistic):              0.393
Time:                        12:59:29   Log-Likelihood:                -2375.6
No. Observations:                3297   AIC:                             4777.
Df Residuals:                    3284   BIC:                             4856.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const                               0.5537      0.016     34.244      0.000       0.522       0.585
treatment_only_f_ambiguity         -0.0428      0.034     -1.256      0.209      -0.110       0.024
treatment_only_f_dehumanization    -0.0372      0.034     -1.087      0.277      -0.104       0.030
treatment_only_f_discrimination    -0.0256      0.035     -0.741      0.458      -0.093       0.042
treatment_only_f_trust              0.0400      0.034      1.184      0.237      -0.026       0.106
treatment_only_v_ambiguity         -0.0250      0.043     -0.583      0.560      -0.109       0.059
treatment_only_v_dehumanization     0.0296      0.046      0.640      0.522      -0.061       0.120
treatment_only_v_discrimination     0.0102      0.046      0.222      0.825      -0.080       0.101
treatment_only_v_trust             -0.0267      0.042     -0.639      0.523      -0.109       0.055
treatment_v+f+dehumanization       -0.0342      0.043     -0.790      0.430      -0.119       0.051
treatment_v+f_ambiguity            -0.0370      0.041     -0.914      0.361      -0.116       0.042
treatment_v+f_discriminacion       -0.0937      0.044     -2.140      0.032      -0.180      -0.008
treatment_v+f_trust                 0.0235      0.041      0.572      0.567      -0.057       0.104
==============================================================================
Omnibus:                    11883.262   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              541.239
Skew:                          -0.163   Prob(JB):                    2.96e-118
Kurtosis:                       1.042   Cond. No.                         8.11
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [204]:
est = sm.OLS(Y_2, X2)
est2 = est.fit()
est2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               OLS Regression Results                               
====================================================================================
Dep. Variable:     education_level_Posgrado   R-squared:                       0.002
Model:                                  OLS   Adj. R-squared:                 -0.002
Method:                       Least Squares   F-statistic:                    0.4716
Date:                      Fri, 29 Apr 2022   Prob (F-statistic):              0.932
Time:                              12:59:29   Log-Likelihood:                -392.65
No. Observations:                      3297   AIC:                             811.3
Df Residuals:                          3284   BIC:                             890.6
Df Model:                                12                                         
Covariance Type:                  nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const                               0.0821      0.009      9.266      0.000       0.065       0.099
treatment_only_f_ambiguity         -0.0205      0.019     -1.098      0.272      -0.057       0.016
treatment_only_f_dehumanization    -0.0198      0.019     -1.058      0.290      -0.057       0.017
treatment_only_f_discrimination     0.0040      0.019      0.213      0.831      -0.033       0.041
treatment_only_f_trust              0.0098      0.018      0.528      0.597      -0.026       0.046
treatment_only_v_ambiguity          0.0134      0.024      0.571      0.568      -0.033       0.060
treatment_only_v_dehumanization    -0.0215      0.025     -0.847      0.397      -0.071       0.028
treatment_only_v_discrimination     0.0156      0.025      0.619      0.536      -0.034       0.065
treatment_only_v_trust              0.0077      0.023      0.337      0.736      -0.037       0.053
treatment_v+f+dehumanization        0.0023      0.024      0.097      0.922      -0.044       0.049
treatment_v+f_ambiguity             0.0123      0.022      0.556      0.578      -0.031       0.056
treatment_v+f_discriminacion       -0.0021      0.024     -0.088      0.930      -0.049       0.045
treatment_v+f_trust                -0.0078      0.022     -0.348      0.728      -0.052       0.036
==============================================================================
Omnibus:                     2013.631   Durbin-Watson:                   1.996
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            12701.034
Skew:                           3.064   Prob(JB):                         0.00
Kurtosis:                      10.410   Cond. No.                         8.11
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [205]:
est = sm.OLS(Y_3, X2)
est2 = est.fit()
est2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               OLS Regression Results                               
====================================================================================
Dep. Variable:     education_level_Primaria   R-squared:                       0.003
Model:                                  OLS   Adj. R-squared:                 -0.000
Method:                       Least Squares   F-statistic:                    0.9515
Date:                      Fri, 29 Apr 2022   Prob (F-statistic):              0.494
Time:                              12:59:29   Log-Likelihood:                -74.928
No. Observations:                      3297   AIC:                             175.9
Df Residuals:                          3284   BIC:                             255.2
Df Model:                                12                                         
Covariance Type:                  nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const                               0.0705      0.008      8.764      0.000       0.055       0.086
treatment_only_f_ambiguity          0.0128      0.017      0.755      0.450      -0.020       0.046
treatment_only_f_dehumanization     0.0101      0.017      0.591      0.555      -0.023       0.043
treatment_only_f_discrimination    -0.0031      0.017     -0.181      0.856      -0.037       0.031
treatment_only_f_trust              0.0178      0.017      1.061      0.289      -0.015       0.051
treatment_only_v_ambiguity         -0.0132      0.021     -0.618      0.537      -0.055       0.029
treatment_only_v_dehumanization    -0.0175      0.023     -0.759      0.448      -0.063       0.028
treatment_only_v_discrimination    -0.0254      0.023     -1.107      0.268      -0.070       0.020
treatment_only_v_trust             -0.0166      0.021     -0.799      0.424      -0.057       0.024
treatment_v+f+dehumanization       -0.0186      0.022     -0.862      0.389      -0.061       0.024
treatment_v+f_ambiguity            -0.0372      0.020     -1.845      0.065      -0.077       0.002
treatment_v+f_discriminacion       -0.0172      0.022     -0.789      0.430      -0.060       0.026
treatment_v+f_trust                -0.0191      0.020     -0.936      0.349      -0.059       0.021
==============================================================================
Omnibus:                     2310.030   Durbin-Watson:                   1.940
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            20934.538
Skew:                           3.484   Prob(JB):                         0.00
Kurtosis:                      13.191   Cond. No.                         8.11
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

El grupo V+F - Ambiguedad,  tiene menos personas con educacion primaria que el control (significancia del 10%)

In [206]:
est = sm.OLS(Y_4, X2)
est2 = est.fit()
est2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               OLS Regression Results                               
====================================================================================
Dep. Variable:     social_stratum_Estrato 6   R-squared:                       0.002
Model:                                  OLS   Adj. R-squared:                 -0.001
Method:                       Least Squares   F-statistic:                    0.6705
Date:                      Fri, 29 Apr 2022   Prob (F-statistic):              0.781
Time:                              12:59:29   Log-Likelihood:                 1238.0
No. Observations:                      3297   AIC:                            -2450.
Df Residuals:                          3284   BIC:                            -2371.
Df Model:                                12                                         
Covariance Type:                  nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const                               0.0316      0.005      5.844      0.000       0.021       0.042
treatment_only_f_ambiguity         -0.0062      0.011     -0.546      0.585      -0.029       0.016
treatment_only_f_dehumanization    -0.0096      0.011     -0.839      0.401      -0.032       0.013
treatment_only_f_discrimination     0.0059      0.012      0.509      0.611      -0.017       0.028
treatment_only_f_trust             -0.0068      0.011     -0.607      0.544      -0.029       0.015
treatment_only_v_ambiguity          0.0130      0.014      0.906      0.365      -0.015       0.041
treatment_only_v_dehumanization    -0.0164      0.015     -1.062      0.288      -0.047       0.014
treatment_only_v_discrimination    -0.0165      0.015     -1.073      0.283      -0.047       0.014
treatment_only_v_trust             -0.0196      0.014     -1.403      0.161      -0.047       0.008
treatment_v+f+dehumanization        0.0009      0.014      0.061      0.951      -0.027       0.029
treatment_v+f_ambiguity             0.0018      0.014      0.130      0.897      -0.025       0.028
treatment_v+f_discriminacion       -0.0116      0.015     -0.791      0.429      -0.040       0.017
treatment_v+f_trust                 0.0084      0.014      0.615      0.539      -0.018       0.035
==============================================================================
Omnibus:                     3512.013   Durbin-Watson:                   2.020
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           140775.757
Skew:                           5.645   Prob(JB):                         0.00
Kurtosis:                      32.955   Cond. No.                         8.11
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [207]:
est = sm.OLS(Y_5, X2)
est2 = est.fit()
est2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               OLS Regression Results                               
====================================================================================
Dep. Variable:     social_stratum_Estrato 1   R-squared:                       0.009
Model:                                  OLS   Adj. R-squared:                  0.006
Method:                       Least Squares   F-statistic:                     2.565
Date:                      Fri, 29 Apr 2022   Prob (F-statistic):            0.00220
Time:                              12:59:29   Log-Likelihood:                -1769.6
No. Observations:                      3297   AIC:                             3565.
Df Residuals:                          3284   BIC:                             3645.
Df Model:                                12                                         
Covariance Type:                  nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const                               0.2442      0.013     18.151      0.000       0.218       0.271
treatment_only_f_ambiguity          0.0022      0.028      0.076      0.939      -0.053       0.058
treatment_only_f_dehumanization     0.0488      0.028      1.715      0.086      -0.007       0.105
treatment_only_f_discrimination    -0.0045      0.029     -0.157      0.875      -0.061       0.052
treatment_only_f_trust             -0.0322      0.028     -1.146      0.252      -0.087       0.023
treatment_only_v_ambiguity         -0.0404      0.036     -1.131      0.258      -0.110       0.030
treatment_only_v_dehumanization    -0.0700      0.039     -1.816      0.069      -0.145       0.006
treatment_only_v_discrimination    -0.1014      0.038     -2.640      0.008      -0.177      -0.026
treatment_only_v_trust             -0.0286      0.035     -0.823      0.410      -0.097       0.040
treatment_v+f+dehumanization       -0.1014      0.036     -2.814      0.005      -0.172      -0.031
treatment_v+f_ambiguity            -0.0942      0.034     -2.795      0.005      -0.160      -0.028
treatment_v+f_discriminacion       -0.0442      0.036     -1.213      0.225      -0.116       0.027
treatment_v+f_trust                -0.0156      0.034     -0.458      0.647      -0.083       0.051
==============================================================================
Omnibus:                      588.736   Durbin-Watson:                   1.959
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              957.677
Skew:                           1.316   Prob(JB):                    1.10e-208
Kurtosis:                       2.783   Cond. No.                         8.11
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

El grupo V+F - Deshumanizacion y V+F Ambiguedad y Solo Video - Discriminacion,  tiene menos personas en estrato 1 que el control (significancia del 5%)

In [208]:
est = sm.OLS(Y_6, X2)
est2 = est.fit()
est2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                              
==================================================================================
Dep. Variable:     department_Bogotá D.C.   R-squared:                       0.002
Model:                                OLS   Adj. R-squared:                 -0.001
Method:                     Least Squares   F-statistic:                    0.6636
Date:                    Fri, 29 Apr 2022   Prob (F-statistic):              0.788
Time:                            12:59:29   Log-Likelihood:                -2365.4
No. Observations:                    3297   AIC:                             4757.
Df Residuals:                        3284   BIC:                             4836.
Df Model:                              12                                         
Covariance Type:                nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const                               0.5589      0.016     34.677      0.000       0.527       0.591
treatment_only_f_ambiguity          0.0280      0.034      0.824      0.410      -0.039       0.095
treatment_only_f_dehumanization    -0.0425      0.034     -1.245      0.213      -0.109       0.024
treatment_only_f_discrimination    -0.0159      0.034     -0.461      0.645      -0.083       0.052
treatment_only_f_trust              0.0064      0.034      0.191      0.849      -0.060       0.072
treatment_only_v_ambiguity          0.0079      0.043      0.185      0.853      -0.076       0.092
treatment_only_v_dehumanization     0.0092      0.046      0.200      0.841      -0.081       0.100
treatment_only_v_discrimination    -0.0026      0.046     -0.056      0.956      -0.093       0.088
treatment_only_v_trust             -0.0140      0.042     -0.337      0.736      -0.096       0.068
treatment_v+f+dehumanization        0.0060      0.043      0.139      0.890      -0.079       0.091
treatment_v+f_ambiguity             0.0022      0.040      0.054      0.957      -0.077       0.081
treatment_v+f_discriminacion        0.0811      0.044      1.857      0.063      -0.005       0.167
treatment_v+f_trust                -0.0275      0.041     -0.673      0.501      -0.108       0.053
==============================================================================
Omnibus:                    12085.056   Durbin-Watson:                   1.997
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              544.796
Skew:                          -0.239   Prob(JB):                    5.00e-119
Kurtosis:                       1.067   Cond. No.                         8.11
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

El grupo de V+F - Discriminacion tiene mas personas de Bogota que el control. 

**Conclusiones**: El grupo V+F - Deshumanizacion y V+F - Discriminacion tienen algunos desbalances en terminos de observables. 

## 2. Percepción sobre noticias: 
Noticias falsas y verdaderas que favorecen a la derecha, a la izquierda o son neutras. 

Posibles respuestas: 1 = Nada Confiable, 5 = Muy Confiable

### 2.1 Noticias falsas que favorecen a la derecha: 

'fake_news_i', 'fake_news_j', 'fake_news_f' 'fake_news_r'

In [209]:
df['fake_right'].describe()

count    3297.000000
mean        2.842053
std         1.069553
min         1.000000
25%         2.000000
50%         3.000000
75%         3.500000
max         5.000000
Name: fake_right, dtype: float64

El puntaje promedio de las 4 noticias falsas que favorecen a la derecha es 2.84, sugiriendo que hay una inclinación del grupo a creer en noticias falsas cuando favorecen a la derecha. 

**Por grupo de tratamiento, si vieron el video, género, nivel educativo y departamento**

In [210]:
video_grp['fake_right'].describe()

,count,mean,std,min,25%,50%,75%,max
video,,,,,,,,
No,2049.0,2.842240,1.073289,1.0,2.0,3.0,3.5,5.0
Sí,1248.0,2.841747,1.063820,1.0,2.0,3.0,3.5,5.0


In [211]:
ttest_ind(no['fake_right'], yes['fake_right'])


Ttest_indResult(statistic=5.637135266564767, pvalue=1.8746495140718232e-08)

El test de diferencias de medias muestra que hay una diferencia estadísticamente significativa entre quienes vieron el video y quienes no. Los que vieron el video son menos suceptibles a noticias falsas que favorecen a la derecha. 

In [212]:
form_grp['fake_right'].describe()

,count,mean,std,min,25%,50%,75%,max
form,,,,,,,,
Confianza,625.0,2.886000,1.083227,1.0,2.00,3.0,3.5,5.0
Deshumanización,559.0,2.869857,1.046281,1.0,2.25,3.0,3.5,5.0
Discriminización,550.0,2.828636,1.105160,1.0,2.00,3.0,3.5,5.0
Tolerancia a la ambigüedad,613.0,2.846656,1.063515,1.0,2.00,3.0,3.5,5.0


In [213]:
treatment_grp['fake_right'].describe()

,count,mean,std,min,25%,50%,75%,max
treatment,,,,,,,,
control,950.0,2.903158,1.095465,1.0,2.00,3.00,3.750,5.0
only_f_ambiguity,276.0,3.003623,1.041191,1.0,2.25,3.00,3.750,5.0
only_f_dehumanization,273.0,2.983516,1.134641,1.0,2.25,3.00,3.750,5.0
only_f_discrimination,267.0,2.926030,0.984564,1.0,2.25,3.00,3.500,5.0
only_f_trust,283.0,2.854240,1.031292,1.0,2.25,3.00,3.500,5.0
only_v_ambiguity,157.0,2.648089,1.046191,1.0,1.75,2.75,3.250,5.0
only_v_dehumanization,132.0,2.625000,1.124205,1.0,1.50,2.75,3.250,5.0
only_v_discrimination,133.0,2.612782,1.049651,1.0,1.75,2.50,3.250,5.0
only_v_trust,167.0,2.844311,1.019987,1.0,2.25,3.00,3.625,5.0


In [214]:
#gender_grp['fake_right'].describe()

In [215]:
education_grp['fake_right'].describe()

,count,mean,std,min,25%,50%,75%,max
education_level,,,,,,,,
Bachillerato,1353.0,2.803585,1.072920,1.0,2.0,3.00,3.50,5.0
Posgrado,267.0,2.888577,1.092950,1.0,2.0,3.00,3.50,5.0
Primaria,217.0,2.779954,1.049813,1.0,2.0,2.75,3.50,5.0
Técnico / Tecnológico,975.0,2.862308,1.058275,1.0,2.0,3.00,3.50,5.0
Título universitario,485.0,2.910825,1.076928,1.0,2.0,3.00,3.75,5.0


In [216]:
#ttest_ind(posgrado['fake_right'], primaria['fake_right'])

Es interesante ver que por nivel educativo, los de posgrado creen mucho menos en las noticias falsas, y los de primaria creen mucho más. Esto lo evidencia también el test de diferencia de medias. 

In [217]:
#department_grp['fake_right'].describe()

### 2.2. Notcias falsas que favorecen a la izquierda

'fake_news_d', 'fake_news_g', 'fake_news_h', 'fake_news_o'

In [218]:
df['fake_left'].describe()

count    3297.00000
mean        2.92296
std         1.04440
min         1.00000
25%         2.25000
50%         3.00000
75%         3.50000
max         5.00000
Name: fake_left, dtype: float64

El puntaje promedio de las 4 noticias falsas que izquierda a la derecha es 2.92, sugiriendo que hay una inclinación del grupo a creer en noticias falsas cuando favorecen a la izqueirda.

**Por grupo de tratamiento, si vieron el video, género, nivel educativo y departamento**

In [219]:
video_grp['fake_left'].describe()

,count,mean,std,min,25%,50%,75%,max
video,,,,,,,,
No,2049.0,2.921181,1.045100,1.0,2.25,3.0,3.75,5.0
Sí,1248.0,2.925881,1.043662,1.0,2.25,3.0,3.50,5.0


In [220]:
ttest_ind(no['fake_left'], yes['fake_left'])

Ttest_indResult(statistic=4.763722173849873, pvalue=1.9825990963857324e-06)

In [221]:
form_grp['fake_left'].describe()

,count,mean,std,min,25%,50%,75%,max
form,,,,,,,,
Confianza,625.0,2.901200,1.060442,1.0,2.25,3.0,3.50,5.0
Deshumanización,559.0,2.921288,1.071931,1.0,2.25,3.0,3.75,5.0
Discriminización,550.0,2.943182,1.080681,1.0,2.25,3.0,3.75,5.0
Tolerancia a la ambigüedad,613.0,2.952284,1.021254,1.0,2.25,3.0,3.50,5.0


In [222]:
treatment_grp['fake_left'].describe()

,count,mean,std,min,25%,50%,75%,max
treatment,,,,,,,,
control,950.0,3.010263,1.059585,1.0,2.25,3.00,3.7500,5.0
only_f_ambiguity,276.0,2.971014,1.059619,1.0,2.25,3.00,3.7500,5.0
only_f_dehumanization,273.0,3.043040,1.095498,1.0,2.50,3.00,3.7500,5.0
only_f_discrimination,267.0,2.960674,0.985252,1.0,2.50,3.00,3.5000,5.0
only_f_trust,283.0,2.919611,0.955079,1.0,2.50,3.00,3.5000,5.0
only_v_ambiguity,157.0,2.807325,1.072258,1.0,2.00,2.75,3.5000,5.0
only_v_dehumanization,132.0,2.844697,1.080946,1.0,2.00,3.00,3.5000,5.0
only_v_discrimination,133.0,2.798872,1.041042,1.0,2.25,3.00,3.5000,5.0
only_v_trust,167.0,2.883234,1.077838,1.0,2.00,3.00,3.5000,5.0


In [223]:
#gender_grp['fake_left'].describe()

In [224]:
#education_grp['fake_left'].describe()

In [225]:
#ttest_ind(posgrado['fake_left'], primaria['fake_left'])

Nuevamente los de posgrado son menos vulnerables a las noticias falsas que los demás.

In [226]:
#department_grp['fake_left'].describe()

### 2.3 Noticias falsas neutras

'fake_news_a', 'fake_news_m'

In [227]:
df['fake_center'].describe()

count    3297.000000
mean        2.791022
std         1.155797
min         1.000000
25%         2.000000
50%         3.000000
75%         3.500000
max         5.000000
Name: fake_center, dtype: float64

El puntaje promedio de las 4 noticias falsas que izquierda a la derecha es 2.79, sugiriendo que hay una inclinación del grupo a creer en noticias falsas neutras.

**Por grupo de tratamiento, si vieron el video, género, nivel educativo y departamento**

In [228]:
form_grp['fake_center'].describe()

,count,mean,std,min,25%,50%,75%,max
form,,,,,,,,
Confianza,625.0,2.797600,1.154455,1.0,2.0,3.0,3.5,5.0
Deshumanización,559.0,2.789803,1.167858,1.0,2.0,3.0,3.5,5.0
Discriminización,550.0,2.831818,1.182903,1.0,2.0,3.0,3.5,5.0
Tolerancia a la ambigüedad,613.0,2.818923,1.139316,1.0,2.0,3.0,3.5,5.0


In [229]:
video_grp['fake_center'].describe()

,count,mean,std,min,25%,50%,75%,max
video,,,,,,,,
No,2049.0,2.780381,1.158817,1.0,2.0,3.0,3.5,5.0
Sí,1248.0,2.808494,1.151071,1.0,2.0,3.0,3.5,5.0


In [230]:
ttest_ind(no['fake_center'], yes['fake_center'])

Ttest_indResult(statistic=5.576835266362474, pvalue=2.646889998591037e-08)

Nuevamente hay una diferencia significativa entre los que vieron el video y los que no. 

In [231]:
treatment_grp['fake_center'].describe()

,count,mean,std,min,25%,50%,75%,max
treatment,,,,,,,,
control,950.0,2.891053,1.160454,1.0,2.000,3.0,3.5,5.0
only_f_ambiguity,276.0,2.826087,1.168763,1.0,2.000,3.0,4.0,5.0
only_f_dehumanization,273.0,2.950549,1.224118,1.0,2.000,3.0,4.0,5.0
only_f_discrimination,267.0,2.874532,1.094324,1.0,2.000,3.0,3.5,5.0
only_f_trust,283.0,2.819788,1.112962,1.0,2.000,3.0,3.5,5.0
only_v_ambiguity,157.0,2.563694,1.151540,1.0,1.500,2.5,3.0,5.0
only_v_dehumanization,132.0,2.746212,1.203519,1.0,1.875,3.0,3.5,5.0
only_v_discrimination,133.0,2.567669,1.065616,1.0,2.000,2.5,3.5,5.0
only_v_trust,167.0,2.721557,1.154620,1.0,2.000,3.0,3.5,5.0


In [232]:
#gender_grp['fake_center'].describe()

In [233]:
#education_grp['fake_center'].describe()

In [234]:
#ttest_ind(posgrado['fake_center'], primaria['fake_center'])

### 2.4 Noticias verdaderas que favorecen a la derecha 

'fake_news_s', 'fake_news_e', 'fake_news_b'

In [235]:
df['true_right'].describe()

count    3297.000000
mean        3.050955
std         0.997823
min         1.000000
25%         2.333333
50%         3.000000
75%         3.666667
max         5.000000
Name: true_right, dtype: float64

La mayoría de las personas creen que la noticia es confiable (y lo es, porque es verdadera)

**Por grupo de tratamiento, si vieron el video, género, nivel educativo y departamento**

In [236]:
form_grp['true_right'].describe()

,count,mean,std,min,25%,50%,75%,max
form,,,,,,,,
Confianza,625.0,3.078400,0.990915,1.0,2.333333,3.0,3.666667,5.0
Deshumanización,559.0,3.077519,0.989970,1.0,2.500000,3.0,3.666667,5.0
Discriminización,550.0,3.035152,1.021514,1.0,2.333333,3.0,3.666667,5.0
Tolerancia a la ambigüedad,613.0,3.036433,1.020992,1.0,2.333333,3.0,3.666667,5.0


In [237]:
video_grp['true_right'].describe()

,count,mean,std,min,25%,50%,75%,max
video,,,,,,,,
No,2049.0,3.039531,1.008541,1.0,2.333333,3.0,3.666667,5.0
Sí,1248.0,3.069712,0.980084,1.0,2.333333,3.0,3.666667,5.0


In [238]:
ttest_ind(no['true_right'], yes['true_right'])

Ttest_indResult(statistic=4.896338557502614, pvalue=1.023339064104128e-06)

En ambos grupos la mayoría de las personas cree que en este grupo de noticias la noticia es verdadera. Sin embargo, el grupo que no vio el video cree que las noticias son más confiables 

In [239]:
treatment_grp['true_right'].describe()

,count,mean,std,min,25%,50%,75%,max
treatment,,,,,,,,
control,950.0,3.135439,1.039843,1.0,2.666667,3.0,3.666667,5.0
only_f_ambiguity,276.0,3.118357,0.977154,1.0,2.333333,3.0,4.000000,5.0
only_f_dehumanization,273.0,3.216117,1.036554,1.0,2.666667,3.0,4.000000,5.0
only_f_discrimination,267.0,3.064919,0.951601,1.0,2.333333,3.0,3.666667,5.0
only_f_trust,283.0,3.008245,0.912402,1.0,2.333333,3.0,3.666667,5.0
only_v_ambiguity,157.0,2.944798,0.961397,1.0,2.333333,3.0,3.333333,5.0
only_v_dehumanization,132.0,2.934343,0.999949,1.0,2.333333,3.0,3.666667,5.0
only_v_discrimination,133.0,2.807018,1.001859,1.0,2.000000,3.0,3.666667,5.0
only_v_trust,167.0,3.067864,0.978715,1.0,2.333333,3.0,3.833333,5.0


In [240]:
#gender_grp['true_right'].describe()

In [241]:
#education_grp['true_right'].describe()

In [242]:
#ttest_ind(posgrado['true_right'], primaria['true_right'])

### 2.5 Noticias verdaderas que favorecen a la izquierda

'fake_news_c', 'fake_news_l', 'fake_news_q'

In [243]:
df['true_left'].describe()

count    3297.000000
mean        3.101203
std         1.001770
min         1.000000
25%         2.333333
50%         3.000000
75%         3.666667
max         5.000000
Name: true_left, dtype: float64

**Por grupo de tratamiento, si vieron el video, género, nivel educativo y departamento**

In [244]:
form_grp['true_left'].describe()

,count,mean,std,min,25%,50%,75%,max
form,,,,,,,,
Confianza,625.0,3.104000,0.999836,1.0,2.666667,3.0,3.666667,5.0
Deshumanización,559.0,3.080501,1.024821,1.0,2.333333,3.0,3.666667,5.0
Discriminización,550.0,3.104242,1.026487,1.0,2.333333,3.0,3.666667,5.0
Tolerancia a la ambigüedad,613.0,3.077216,1.028560,1.0,2.333333,3.0,3.666667,5.0


In [245]:
video_grp['true_left'].describe()

,count,mean,std,min,25%,50%,75%,max
video,,,,,,,,
No,2049.0,3.103790,0.999765,1.0,2.333333,3.0,3.666667,5.0
Sí,1248.0,3.096955,1.005439,1.0,2.666667,3.0,3.666667,5.0


In [246]:
ttest_ind(no['true_left'], yes['true_left'])

Ttest_indResult(statistic=2.4141029871964115, pvalue=0.01582818025916834)

In [247]:
treatment_grp['true_left'].describe()

,count,mean,std,min,25%,50%,75%,max
treatment,,,,,,,,
control,950.0,3.120702,1.032287,1.0,2.333333,3.000000,3.666667,5.0
only_f_ambiguity,276.0,3.170290,0.961263,1.0,2.666667,3.000000,4.000000,5.0
only_f_dehumanization,273.0,3.175824,1.058355,1.0,2.666667,3.000000,4.000000,5.0
only_f_discrimination,267.0,3.157303,0.933132,1.0,2.666667,3.000000,3.666667,5.0
only_f_trust,283.0,3.081272,0.952409,1.0,2.333333,3.000000,3.666667,5.0
only_v_ambiguity,157.0,3.178344,0.946976,1.0,2.666667,3.333333,4.000000,5.0
only_v_dehumanization,132.0,3.176768,0.975040,1.0,2.666667,3.000000,4.000000,5.0
only_v_discrimination,133.0,2.977444,1.070885,1.0,2.333333,3.000000,3.666667,5.0
only_v_trust,167.0,3.095808,0.980466,1.0,2.666667,3.000000,3.666667,5.0


In [248]:
#gender_grp['true_left'].describe()

In [249]:
#education_grp['true_left'].describe()

In [250]:
#ttest_ind(posgrado['true_left'], primaria['true_left'])

### 2.6 Noticias verdaderas que favorecen a la izquierda

'fake_news_k', 'fake_news_p', 'fake_news_n'

In [251]:
df['true_center'].describe()

count    3297.000000
mean        2.871600
std         1.065176
min         1.000000
25%         2.000000
50%         3.000000
75%         3.666667
max         5.000000
Name: true_center, dtype: float64

In [252]:
form_grp['true_center'].describe()

,count,mean,std,min,25%,50%,75%,max
form,,,,,,,,
Confianza,625.0,2.923733,1.066376,1.0,2.333333,3.0,3.666667,5.0
Deshumanización,559.0,2.865236,1.076396,1.0,2.000000,3.0,3.666667,5.0
Discriminización,550.0,2.893333,1.099178,1.0,2.000000,3.0,3.666667,5.0
Tolerancia a la ambigüedad,613.0,2.853725,1.070072,1.0,2.000000,3.0,3.666667,5.0


In [253]:
video_grp['true_center'].describe()

,count,mean,std,min,25%,50%,75%,max
video,,,,,,,,
No,2049.0,2.859769,1.063198,1.0,2.0,3.0,3.666667,5.0
Sí,1248.0,2.891026,1.068559,1.0,2.0,3.0,3.666667,5.0


In [254]:
ttest_ind(no['true_center'], yes['true_center'])

Ttest_indResult(statistic=5.055827394540874, pvalue=4.519100871950476e-07)

In [255]:
treatment_grp['true_left'].describe()

,count,mean,std,min,25%,50%,75%,max
treatment,,,,,,,,
control,950.0,3.120702,1.032287,1.0,2.333333,3.000000,3.666667,5.0
only_f_ambiguity,276.0,3.170290,0.961263,1.0,2.666667,3.000000,4.000000,5.0
only_f_dehumanization,273.0,3.175824,1.058355,1.0,2.666667,3.000000,4.000000,5.0
only_f_discrimination,267.0,3.157303,0.933132,1.0,2.666667,3.000000,3.666667,5.0
only_f_trust,283.0,3.081272,0.952409,1.0,2.333333,3.000000,3.666667,5.0
only_v_ambiguity,157.0,3.178344,0.946976,1.0,2.666667,3.333333,4.000000,5.0
only_v_dehumanization,132.0,3.176768,0.975040,1.0,2.666667,3.000000,4.000000,5.0
only_v_discrimination,133.0,2.977444,1.070885,1.0,2.333333,3.000000,3.666667,5.0
only_v_trust,167.0,3.095808,0.980466,1.0,2.666667,3.000000,3.666667,5.0


In [256]:
#gender_grp['true_center'].describe()

In [257]:
#education_grp['true_center'].describe()

In [258]:
#ttest_ind(posgrado['true_center'], primaria['true_center'])

## 3. Percepción sobre tweets: 
Dos tweets, dos preguntas: 1. El tweet es inapropiado. 2. Yo denunciaria este tweet. 

Posibles respuestas: 1 = Si, 0 = No

In [259]:
df[['tweet_a_lack_of_information', 'tweet_a_report', 'tweet_b_lack_of_information', 'tweet_b_report']].describe()


,tweet_a_lack_of_information,tweet_a_report,tweet_b_lack_of_information,tweet_b_report
count,3297.000000,3297.000000,3297.000000,3297.000000
mean,0.742796,0.631180,0.711859,0.599029
std,0.437159,0.482558,0.452966,0.490169
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,1.000000,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000


In [260]:
treatment_grp[['tweet_a_lack_of_information', 'tweet_a_report', 'tweet_b_lack_of_information', 'tweet_b_report']].mean()

,tweet_a_lack_of_information,tweet_a_report,tweet_b_lack_of_information,tweet_b_report
treatment,,,,
control,0.730526,0.592632,0.700000,0.581053
only_f_ambiguity,0.757246,0.644928,0.655797,0.594203
only_f_dehumanization,0.717949,0.622711,0.717949,0.582418
only_f_discrimination,0.707865,0.647940,0.677903,0.614232
only_f_trust,0.742049,0.621908,0.763251,0.611307
only_v_ambiguity,0.802548,0.687898,0.738854,0.681529
only_v_dehumanization,0.795455,0.621212,0.704545,0.522727
only_v_discrimination,0.721805,0.646617,0.751880,0.661654
only_v_trust,0.754491,0.670659,0.664671,0.622754


## 4. Preguntas Sobre Desconfianza

- Primeras tres preguntas: 1 = Nada Confiable, 4 = Muy confiable 
    1. Personas del barrio confiables. 
    2. Personas del pais confiables. 
    3. Personas de la ciudad confiables. 
    
    
- Siguientes dos preguntas: 1 = Si, 0 = No 
    1. Se puede confiar en la gente. 
    2. Funcionarios publicos pueden ser sobornados facilmente. 


- Siguientes 4 preguntas: 1 = Nada, 7 = Mucho
     Cumplen lo que han prometido: 
     1. Rama Judicial 
     2. Funcionaros Publicos 
     3. Politicos 
     4. Empresarios 

### 4.1 Regresion con Y = Personas del Pais son confiables

In [261]:
Y_1 = df_dummies['trust_2']
Y_2 = df_dummies['trust_5']
Y_3 = df_dummies['trust_8']

In [262]:
est = sm.OLS(Y_1, X2)
est2 = est.fit()
est2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                trust_2   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.406
Date:                Fri, 29 Apr 2022   Prob (F-statistic):              0.155
Time:                        14:27:08   Log-Likelihood:                -4967.8
No. Observations:                3297   AIC:                             9962.
Df Residuals:                    3284   BIC:                         1.004e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const                               3.1253      0.035     88.053      0.000       3.056       3.195
treatment_only_f_ambiguity          0.0124      0.075      0.166      0.868      -0.134       0.159
treatment_only_f_dehumanization     0.0982      0.075      1.307      0.191      -0.049       0.245
treatment_only_f_discrimination     0.0245      0.076      0.324      0.746      -0.124       0.173
treatment_only_f_trust             -0.0581      0.074     -0.785      0.433      -0.203       0.087
treatment_only_v_ambiguity         -0.1444      0.094     -1.532      0.126      -0.329       0.040
treatment_only_v_dehumanization    -0.0344      0.102     -0.338      0.735      -0.234       0.165
treatment_only_v_discrimination    -0.1553      0.101     -1.534      0.125      -0.354       0.043
treatment_only_v_trust              0.1682      0.092      1.832      0.067      -0.012       0.348
treatment_v+f+dehumanization        0.1215      0.095      1.278      0.201      -0.065       0.308
treatment_v+f_ambiguity            -0.0530      0.089     -0.596      0.551      -0.227       0.121
treatment_v+f_discriminacion        0.0214      0.096      0.223      0.824      -0.167       0.210
treatment_v+f_trust                 0.1205      0.090      1.338      0.181      -0.056       0.297
==============================================================================
Omnibus:                       54.628   Durbin-Watson:                   1.964
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               37.548
Skew:                          -0.143   Prob(JB):                     7.02e-09
Kurtosis:                       2.562   Cond. No.                         8.11
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Los resultados muestran que solo V de confianza sirven para que las personas confien mas en las personas del pais. Sin embargo, F solo no sirve y V+F tampoco.  

### 4.2 Regresion con Y = Funcionarios del Sector Publico pueden ser sobornados facilmente

In [263]:
est = sm.OLS(Y_2, X2)
est2 = est.fit()
est2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                trust_5   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.286
Date:                Fri, 29 Apr 2022   Prob (F-statistic):              0.220
Time:                        14:27:08   Log-Likelihood:                -1919.9
No. Observations:                3297   AIC:                             3866.
Df Residuals:                    3284   BIC:                             3945.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const                               0.7463      0.014     52.999      0.000       0.719       0.774
treatment_only_f_ambiguity         -0.0507      0.030     -1.707      0.088      -0.109       0.008
treatment_only_f_dehumanization     0.0156      0.030      0.523      0.601      -0.043       0.074
treatment_only_f_discrimination    -0.0235      0.030     -0.781      0.435      -0.082       0.035
treatment_only_f_trust             -0.0219      0.029     -0.746      0.456      -0.080       0.036
treatment_only_v_ambiguity          0.0308      0.037      0.822      0.411      -0.043       0.104
treatment_only_v_dehumanization     0.0794      0.040      1.970      0.049       0.000       0.158
treatment_only_v_discrimination     0.0657      0.040      1.635      0.102      -0.013       0.145
treatment_only_v_trust              0.0022      0.036      0.060      0.952      -0.069       0.074
treatment_v+f+dehumanization        0.0329      0.038      0.873      0.383      -0.041       0.107
treatment_v+f_ambiguity            -0.0241      0.035     -0.683      0.495      -0.093       0.045
treatment_v+f_discriminacion        0.0137      0.038      0.359      0.720      -0.061       0.088
treatment_v+f_trust                 0.0023      0.036      0.063      0.950      -0.068       0.072
==============================================================================
Omnibus:                      636.341   Durbin-Watson:                   1.963
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              771.728
Skew:                          -1.134   Prob(JB):                    2.64e-168
Kurtosis:                       2.309   Cond. No.                         8.11
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Ninguno de los tratamientos de confianza afectan la respuesta a esta pregunta. 

### 4.3 Regresion con Y = Politicos cumplen con lo que han prometido

In [264]:
est = sm.OLS(Y_3, X2)
est2 = est.fit()
est2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                trust_8   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     3.142
Date:                Fri, 29 Apr 2022   Prob (F-statistic):           0.000182
Time:                        14:27:08   Log-Likelihood:                -5536.7
No. Observations:                3297   AIC:                         1.110e+04
Df Residuals:                    3284   BIC:                         1.118e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const                               2.6105      0.042     61.894      0.000       2.528       2.693
treatment_only_f_ambiguity          0.1974      0.089      2.221      0.026       0.023       0.372
treatment_only_f_dehumanization     0.1441      0.089      1.614      0.107      -0.031       0.319
treatment_only_f_discrimination     0.0374      0.090      0.415      0.678      -0.139       0.214
treatment_only_f_trust             -0.0416      0.088     -0.473      0.636      -0.214       0.131
treatment_only_v_ambiguity         -0.2156      0.112     -1.925      0.054      -0.435       0.004
treatment_only_v_dehumanization    -0.1030      0.121     -0.853      0.394      -0.340       0.134
treatment_only_v_discrimination    -0.4526      0.120     -3.761      0.000      -0.689      -0.217
treatment_only_v_trust             -0.0357      0.109     -0.327      0.744      -0.250       0.178
treatment_v+f+dehumanization       -0.0975      0.113     -0.864      0.388      -0.319       0.124
treatment_v+f_ambiguity            -0.1883      0.106     -1.782      0.075      -0.396       0.019
treatment_v+f_discriminacion       -0.1772      0.114     -1.551      0.121      -0.401       0.047
treatment_v+f_trust                 0.0352      0.107      0.329      0.742      -0.174       0.245
==============================================================================
Omnibus:                      668.749   Durbin-Watson:                   1.938
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              180.458
Skew:                           0.309   Prob(JB):                     6.52e-40
Kurtosis:                       2.035   Cond. No.                         8.11
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Los tratamientos de confianza no afectan en la respuesta a esta pregunta. Curiosamente, aquellos que vieron el video de discriminacion creen significamente menos en que los politicos cumplen lo que prometen

## 5. Discriminación 

6 preguntas con respuestas 1 = Casi todos los dias, 2 = Al menos una vez a la semana, 3 = Algunas veces al mes, 4 = Algunas veces al año 5 = Nunca. 

- Preguntas 
    1. Te tratan con menos cortesia que a otros debido a tu ideologia
    2. Te tratan con menos respeto que a otros debido a tu ideologia
    3. Actuan como si no fueras inteligente debido a tu ideologia
    4. Actuan como si te tuvieran miedo debido a tu ideologia
    5. Actuan como si fueras deshonesto debido a tu ideologia
    6. Actuan como si fueran mejores que tu debido a tu ideologia
    
 Pregunta: Como estas respuestas se podrian ver modificadas por los tratamientos, no se buscaba medir discriminacion hacia los demas? 


## 6. Necesidad de Cierre Cognitivo (tolerancia a la ambigüedad) 

Respuestas: 1 = Muy en desacuerdo, 5 = Muy de acuerdo. 

- Preguntas: 
    1. Vida ordenada con horarios establecidos 
    2. Odia cambiar planes al ultimo minuto 
    3. No le gustan las situaciones inciertas. 
    4. Incomodo cuando no entiende por que paso un evento en su vida. 
    5. Aliviado cuando toma una decision
    6. Casi siempre con afan para tomar una decision, incluso cuando no hay afan. 
    7. Cuando lo confrontan ansioso por llegar a una solucion rapido. 
    8. Cuando va a un restaurante le gusta ir a donde ya ha estado. 
    9. Le gusta tener amigos impredescibles (codificada al reves) 
    10. Incluso cuando ha tomado una decision, le emociona considerar una opcion distinta. 
